In [5]:
import sys
sys.path.append('.')

import numpy as np

from annotation_ems.gen_dataset import generate_dataset
from annotation_ems.ems_prepare import split, prepare_json
import itertools as it

import subprocess
import os
import time
import random

### begin of config

dataset_path = '/mnt/data/jarvislam1999'
output_path = './annotation_ems/'
model =  '/mnt/data/jarvislam1999/Real-time-GesRec/results/ems25.1/save_30.pth'
n_classes = 4

modality = "rgb"  # d, rgb, rgbd
labels = ['wrist_left', 'wrist_right', 'pronation', 'supination']

In [6]:
data_batch = ['12_1_basic_bg0001', \
             '12_1_basic_bg0002', \
             '12_1_basic', \
             '12_2_basic2_depth_bg0001', \
             '12_2_basic2_depth_bg0002', \
             '12_2_basic2', \
             '12_4_position2_bg0001',\
             '12_4_position2_bg0002',\
             '12_4_position2',\
             'subject01_setting3_bg03_01',\
             'subject01_setting3_bg04_01']
#data_batch = [6,5,4,3,2,1]
data_batch.sort()

In [7]:
train_ratio = 50
test_ratio = 50
count = 1
expr_name = "27.8"

train_partition = {x: train_ratio for x in data_batch}
test_partition = {x: test_ratio for x in data_batch}
generate_dataset(expr_name=expr_name, modality=modality, dataset_path=dataset_path, output_path=output_path, train_partition=train_partition, test_partition=test_partition, labels=labels)
prepare_json(csv_dir_path=output_path, expr_name=expr_name)
print(expr_name)


27.8


In [11]:
processes = set()

cmd = "CUDA_VISIBLE_DEVICES=%d nohup python main.py \
        --root_path /mnt/data/jarvislam1999/Real-time-GesRec \
        --video_path %s \
        --annotation_path annotation_ems/ems%s.json\
        --result_path results/ems%s \
        --pretrain_path %s \
        --dataset ems \
        --sample_duration 32 \
        --learning_rate 0.01 \
        --model resnext \
        --model_depth 101 \
        --resnet_shortcut B \
        --batch_size 12 \
        --n_classes %d \
        --n_finetune_classes 4 \
        --n_threads 16 \
        --checkpoint 1 \
        --modality RGB \
        --train_crop random \
        --n_val_samples 1 \
        --test_subset test \
        --n_epochs 40 \
        --no_val \
        --checkpoint 5\
    " % ((count-1) % 4, dataset_path, expr_name,expr_name, model, n_classes)
print(expr_name)
out = open('ems%sout.txt'%(expr_name), 'w')
processes.add(subprocess.Popen(cmd, shell=True, stdout = out, stderr = out))
#Check if all the child processes were closed
for p in processes:
    if p.poll() is None:
        p.wait()
        


27.8


In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "%d" % random.randrange(0,4)

import sys
# sys.path.append('../')
os.chdir("/mnt/data/jarvislam1999/Real-time-GesRec/")

from ems_tester import EMSTester

In [14]:
tester = EMSTester(root_path='/mnt/data/jarvislam1999/Real-time-GesRec',
          video_path='%s'% dataset_path,
          annotation_path='annotation_ems/ems%s.json' % (expr_name),
          result_path='results/ems%s_test' % (expr_name),
          model_path='results/ems%s/save_30.pth' %(expr_name))
y_pred, y_true, test_data = tester.test()


Namespace(annotation_path='/mnt/data/jarvislam1999/Real-time-GesRec/annotation_ems/ems27.8.json', arch='resnext-101', batch_size=1, begin_epoch=1, checkpoint=1, crop_position_in_test='c', dampening=0.9, dataset='ems', ft_begin_index=0, initial_scale=1.0, learning_rate=0.1, lr_patience=10, lr_steps=[10, 25, 50, 80, 100], manual_seed=1, mean=[114.7748, 107.7354, 99.475], mean_dataset='activitynet', modality='RGB', model='resnext', model_depth=101, momentum=0.9, n_classes=400, n_epochs=200, n_finetune_classes=4, n_scales=5, n_threads=1, n_val_samples=1, nesterov=False, no_cuda=False, no_hflip=False, no_mean_norm=False, no_softmax_in_test=False, no_train=False, no_val=False, norm_value=1, optimizer='sgd', pretrain_path='', resnet_shortcut='B', resnext_cardinality=32, result_path='/mnt/data/jarvislam1999/Real-time-GesRec/results/ems27.8_test', resume_path='/mnt/data/jarvislam1999/Real-time-GesRec/results/ems27.8/save_30.pth', root_path='/mnt/data/jarvislam1999/Real-time-GesRec', sample_dura